<a href="https://colab.research.google.com/github/CarlosG-CDU/GroupAssignment-2/blob/main/Assignment2_Q2_Updated_CO_CG_Functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#added functons
#Group Assignment 2
#DAN/EXT 05 - Group 20
#Question 2
#Carlos Galli S365408
#Lauren Whitford S373432
#Cody Old S392352



#import pandas
#csvFile = pandas.read_csv('/content/stations_group_1986.csv')
#print(csvFile)


import os
import pandas as pd
import glob
import os
import re


def main():
  combine_df = joinData()
  seasonalAvg(combine_df)
  tempRange(combine_df)
  tempStability(combine_df)

##############################
#Join all the data together in one big bucket

def joinData():
  csv_files = glob.glob('/content/temperatures/*.csv')  #ignore the year at the end
  if not csv_files:
        print("Error: No CSV files found in /content/temperatures/")
        return pd.DataFrame()  # Return empty DataFrame if no files

  combine_df = pd.DataFrame()

  for csv_file in csv_files:
      filename = os.path.basename(csv_file)   #get the file name
      #print(filename)
##############################
#find the year in file name and place in new colum for each data block

      pattern = r'\d{4}'
      match = re.search(pattern, filename)
      if match:
          year = match.group()
      else:
          year = None
      #print(year)

      df = pd.read_csv(csv_file)  #read and store the the csv file to add the year column
      df["Year"] = year   # place each year in the "YEAR" column
      combine_df = pd.concat([combine_df, df], ignore_index = True) #add to combined DataFrame

  #Sort by Year fwiw
  combine_df = combine_df.sort_values(by = "Year").reset_index(drop=True)

  combine_df.to_csv("/content/output.csv", index = False)   #use to check contents. remove later
  print("saved to output.csv")


  return combine_df #return the combined Data Frame

################################
#seasonal avg for all stations and years

def seasonalAvg(combine_df):
  seasonal_avg = pd.DataFrame()
  seasonal_avg["Year"] = combine_df["Year"]

  summer_values = pd.concat([combine_df['December'], combine_df['January'], combine_df['February']])
  summer_avg = summer_values.mean().round(1)

  autumn_values = pd.concat([combine_df['March'], combine_df['April'], combine_df['May']])
  autumn_avg = autumn_values.mean().round(1)

  winter_values = pd.concat([combine_df['June'], combine_df['July'], combine_df['August']])
  winter_avg = winter_values.mean().round(1)

  spring_values = pd.concat([combine_df['September'], combine_df['October'], combine_df['November']])
  spring_avg = spring_values.mean().round(1)


  #print(f"Summer: {summer_avg}°C\n")  #debug
  #print(f"Autumn: {autumn_avg}°C\n")  #debug
  #print(f"Winter: {winter_avg}°C\n")  #debug
  #print(f"Spring: {spring_avg}°C\n")  #debug

  with open('/content/average_temp.txt', 'w') as f:   #save avg's to .txt as requested by Q1
      f.write(f"Summer: {summer_avg}°C\n")
      f.write(f"Autumn: {autumn_avg}°C\n")
      f.write(f"Winter: {winter_avg}°C\n")
      f.write(f"Spring: {spring_avg}°C\n")

################################
# Temperature Range: Largest range per station

def tempRange(combine_df):
  # List of month columns in your data
  month_cols = ['January','February','March','April','May','June',
                'July','August','September','October','November','December']

  # Group by station and find overall max and min across all months
  #Ref: https://pandas.pydata.org/pandas-docs/stable//reference/api/pandas.core.groupby.DataFrameGroupBy.max.html
  #Ref: https://www.statology.org/pandas-max-across-columns/
  station_max = combine_df.groupby('STATION_NAME')[month_cols].max().max(axis=1)
  station_min = combine_df.groupby('STATION_NAME')[month_cols].min().min(axis=1)

  # Calculate range
  station_range = station_max - station_min

  # Find the largest range value
  max_range_value = station_range.max()

  # Get all stations with that range
  largest_range_stations = station_range[station_range == max_range_value].index #This includes any ties

  # Save results to file
  with open('/content/largest_temp_range_station.txt', 'w') as f:
      for station in largest_range_stations:
          f.write(f"{station}: Range {max_range_value:.1f}°C "
                  f"(Max: {station_max[station]:.1f}°C, Min: {station_min[station]:.1f}°C)\n")

  print("Saved largest temperature range results to largest_temp_range_station.txt")

################################
# Temperature Stability: Most stable and most variable stations
#Ref: https://www.geeksforgeeks.org/python/pandas-multi-index-and-groupby/

def tempStability(combine_df):

  month_cols = ['January','February','March','April','May','June',
                'July','August','September','October','November','December']
  # Keep only the station name and month columns
  df_months = combine_df[['STATION_NAME'] + month_cols]

  # Set the station name as the index
  df_months = df_months.set_index('STATION_NAME')

  # Flatten all months into one column per station, then calculate std dev
  station_overall_std = df_months.stack().groupby(level=0).std()

  # Find the smallest and largest std dev values
  min_std_value = station_overall_std.min()
  max_std_value = station_overall_std.max()

  # Get the station names for most stable and most variable
  most_stable_stations = station_overall_std[station_overall_std == min_std_value].index
  most_variable_stations = station_overall_std[station_overall_std == max_std_value].index

  # Save results to file
  with open('/content/temperature_stability_stations.txt', 'w') as f:
      for station in most_stable_stations:
          f.write(f"Most Stable: {station}: StdDev {min_std_value:.1f}°C\n")
      for station in most_variable_stations:
          f.write(f"Most Variable: {station}: StdDev {max_std_value:.1f}°C\n")

  print("Saved temperature stability results to temperature_stability_stations.txt")


main()

saved to output.csv
Saved largest temperature range results to largest_temp_range_station.txt
Saved temperature stability results to temperature_stability_stations.txt
